## 11 - How to visualise data with xcube viewer in your workspace
### A DeepESDL example notebook 

This notebook demonstrates how to use the features of the xcube JupyterLab integration.
The notebook demonstrates three scenarios how xcube Viewer is utilized in JupyterLab.
In particular, we open xcube Viewer for any `xarray.Dataset` instances 

1. persisted in team s3 storage (saved datasets)
2. opened or otherwise created in this Notebook (in-memory datasets)
3. using a configuration file for customising stlyes. 

To explore the xcube viewer functionalities, please checkout the documentation: https://xcube.readthedocs.io/en/latest/viewer.html#functionality

Please, also refer to the [DeepESDL documentation](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/) and visit the platform's [website](https://www.earthsystemdatalab.net/) for further information!

Brockmann Consult, 2023

-----------------

**This notebook runs with the python environment `deepesdl-xcube-1.1.2`, please checkout the documentation for [help on changing the environment](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kerne).**

First, lets create a small cube, which we can visualise with xcube viewer. We will use ESA CCI data for this. Please head over to 03 - Generate CCI data cubes to get more details about the xcube-cci data store :)

In [1]:
from xcube.core.store import new_data_store
import os

In [2]:
store = new_data_store('cciodp')
store

We request a dataset from the datasore:

In [3]:
dataset = store.open_data('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.2-1.sst', 
                          variable_names=['analysed_sst'],
                          time_range=['1981-09-01','1981-09-07'])

dataset

<xarray.Dataset>
Dimensions:       (time: 7, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-01T12:00:00 ... 1981-09-07T12...
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(7, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2023-07-14T08:36:44.826691
    processing_level:        L4
    time_coverage_start:     1981-09-01T00:00:00
    time_coverage_end:       1981-09-08T00:00:00
    time_coverage_duration:  P7DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

Next, save it to the team s3 storage:

To store the cube in your teams user space, please first retrieve the details from your environment variables as the following:

In [4]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

You need to instantiate a s3 datastore pointing to the team bucket:

In [5]:
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))


If you have stored no data to your user space, the returned list will be empty:

In [6]:
list(team_store.get_data_ids())

[]

The writing will take a few moments, as the data is global and will be persisted into the team s3 storage. 

In [7]:
output_id = 'analysed_sst.zarr'

In [8]:
team_store.write_data(dataset, output_id)

'analysed_sst.zarr'

If you list the content of you datastore again, you will now see the newly written dataset in the list:

In [9]:
list(team_store.get_data_ids())

['analysed_sst.zarr']

Once the cube is stored in our team s3 storage, we can use xcube viewer jupyterlab extention to visualise it.

In [10]:
from xcube.webapi.viewer import Viewer

We use the xcube datastore framework here to open the dataset, but it could also be opened by other means, e.g., `xr.open_dataset()`, provided it has variables with dimensions ["time", "y", "x"] or ["y", "x"]. 

In [11]:
dataset = team_store.open_data(output_id)

In [12]:
dataset

<xarray.Dataset>
Dimensions:       (time: 7, lat: 3600, lon: 7200, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.97 -89.93 -89.88 ... 89.88 89.93 89.97
    lat_bnds      (lat, bnds) float32 dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * lon           (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    lon_bnds      (lon, bnds) float32 dask.array<chunksize=(7200, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 1981-09-01T12:00:00 ... 1981-09-07T12...
    time_bnds     (time, bnds) datetime64[ns] dask.array<chunksize=(7, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 1200, 2400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    date_created:            2023-07-14T08:36:44.826691
    history:                 [{'cube_params': {'time_range': ['1981-09-01T00:...
    processing_level:        L4
    time_coverage_duration:  P7DT0H0M0S
    time_coverage_end:       1981-09-08T00:00:00
    time_coverage_start:     1981-09-01T00:00:00
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...

---
**Scenario 1**: Open xcube Viewer for a dataset instances persisted from a certain source (saved datasets).

**For the functionalities of xcube viewer please head over to the documentation:** https://xcube.readthedocs.io/en/latest/viewer.html#functionality

In [13]:
viewer = Viewer()

In [14]:
viewer.add_dataset(dataset)

'a5e7a06d-fe65-43ce-8732-8d56c6085c50'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [15]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8000
Viewer: https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8000/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8000


You can also open xcube Viewer inlined here:

In [16]:
viewer.show()

To stop the server and viewer:

In [17]:
viewer.stop_server()

---
**Scenario 2**: Open xcube Viewer for a dataset instances opened or otherwise created in this Notebook (in-memory datasets).

Below, let's fetch a dataset from CDS on the fly, without persisting it - for more details about xcube CDS datastore please checkout the example notebook 07 GENERATE C3S CDS CUBES. Be aware of performance loss, so in case you plan to use a dataset a lot for analysis or visualisation, please persist it into the team s3 storage. The public cubes provided within DeepESDL are already persisted in S3, so you should not duplicate them in your team storage. 

In [18]:
cds_store = new_data_store('cds')

In [19]:
cds_dataset = cds_store.open_data('reanalysis-era5-single-levels-monthly-means:monthly_averaged_reanalysis', 
                          variable_names=['2m_temperature'], 
                          bbox=[-10, 45, 40, 65], 
                          spatial_res=0.25, 
                          time_range=['2010-01-01', '2010-12-31'])

xcube-cds version 0.9.2
2023-07-14 08:39:18,722 INFO Welcome to the CDS
INFO:cdsapi:Welcome to the CDS
2023-07-14 08:39:18,723 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
INFO:cdsapi:Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-07-14 08:39:18,816 INFO Request is completed
INFO:cdsapi:Request is completed
2023-07-14 08:39:18,818 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data3/adaptor.mars.internal-1689323240.6210227-26692-14-2cd903bf-7cd1-46fb-a077-a4ce5f42f106.nc to /tmp/tmp77p799wi/tmp9tsf_z_m/data (377.2K)
INFO:cdsapi:Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data3/adaptor.mars.internal-1689323240.6210227-26692-14-2cd903bf-7cd1-46fb-a077-a4ce5f42f106.nc to /tmp/tmp77p799wi/tmp9tsf_z_m/data (377.2K)
2023-07-14 08:39:19,059 INFO Download rate 1.5

In [20]:
cds_dataset.attrs["title"] = "ERA5 2m Temperature"


Let's set some attributes of the variable, so the colormapping does not fall back on the default

In [21]:
cds_dataset.t2m.attrs["color_value_min"] = 270
cds_dataset.t2m.attrs["color_value_max"] = 310
cds_dataset.t2m.attrs["color_bar_name"] = "plasma"

In [22]:
viewer = Viewer()

In [23]:
viewer.add_dataset(dataset)
viewer.add_dataset(cds_dataset)

'71f444d9-7d90-4371-8c90-bbc970445c22'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [24]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8001
Viewer: https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8001/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/alicebalfanz/proxy/8001


You can also open xcube Viewer inlined here:

In [25]:
viewer.show()

To stop the server and viewer:

In [26]:
viewer.stop_server()

---
**Scenario 3**: Use custom server configuration to start server and pass it to the viewer constructor. In this case, we have created a local file with the configuration and load it as a dictionary and pass it to the viewer. 

The custom configuration allows you to predefine your value ranges, the colormaps that should be used as well as which bands should be used to create an RGB image, then the RGB switch in the viewer will display the RGB image. 

If you do not have a server-config.yaml file in your directory, please create one with the following content: 

```yaml
DataStores:
  - Identifier: deep-esdl-cci-sst
    StoreId: s3
    StoreParams:
      root: $S3_USER_STORAGE_BUCKET
      storage_options:
          anon: false
          key: $S3_USER_STORAGE_KEY
          secret: $S3_USER_STORAGE_SECRET
    Datasets:
      - Path: "*.zarr"
        Style: default

        # ChunkCacheSize: 1G


Styles:
  - Identifier: default
    ColorMappings:
      analysed_sst:
        ColorBar: plasma
        ValueRange: [270, 310]
## if you have bands that can create an RGB image, you can specify them as below. 
#      rgb:
#        Red:
#          Variable: B04
#          ValueRange: [0., 0.25]
#        Green:
#          Variable: B03
#          ValueRange: [0., 0.25]
#        Blue:
#          Variable: B02
#          ValueRange: [0., 0.25]

```

**For all possible settings within the server configuration file, please checkout the documentation:** https://xcube.readthedocs.io/en/latest/cli/xcube_serve.html#configuration-file

In [27]:
from xcube.util.config import load_configs

In [28]:
viewer = Viewer(server_config=load_configs("server-config.yaml"))

In [29]:
viewer.show()

To stop the server and viewer:

In [30]:
viewer.stop_server()

Afer all our testing and exploring, let's clean up the example cube :) 

In [31]:
team_store.delete_data(output_id)